In [2]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.tech/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [3]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0.5)
    return completion.choices[0].message.content

In [17]:
import json
from tqdm import tqdm

In [19]:
from langchain import hub

prompt = hub.pull("tyfann/llm4commit-rag")

In [20]:
with open('../data/test_data_with_rag_100.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

In [21]:
gpt_msg = []
for data in tqdm(org_data, total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'] + "\nCommit message:"+ data['sim_msg'], "question": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_msg.append(gpt_35_api(example_prompt))

Processing documents: 100%|██████████| 100/100 [02:06<00:00,  1.27s/it]


In [27]:
with open('../data/msg_nngen_nmt_codebert_chatgpt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/chatgpt_rag2.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)